# Task is to Reading mail from Outlook
### Note : Only for first time mail and forwarded.
### 1. Connecting Notebook with Outlook
### 2. Reading mail from category 1 folder, fetching body of mail only and storing into CSV
### 3. Reading mail from category 2 folder, fetching body of mail only and storing into CSV

# Importing Libraries

In [1]:
# Library for reading mail and writing to csv
import unicodecsv as csv
import email
from bs4 import BeautifulSoup
from imap_tools import MailBox, AND, A
import re

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns



# NLP LIBRARIES
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *
from nltk.classify import NaiveBayesClassifier
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


# downloading stopwords corpus
nltk.download('stopwords')
#nltk.download('wordnet')
#stopwords = set(stopwords.words("english"))

# for showing all the plots inline
%matplotlib inline

#!pip install spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mansurya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# For AR folder
#### Fetching first mail from trial mail including from forwarded mails

In [10]:
# Saving mail to inbox_AR for Action required mails
output_file = open(r'C:/Users/mansurya/Desktop/Email_triage/test/cent_inbox_28_12.csv','wb')
#C:/Users/mansurya/Desktop/Email_triage/EmailTriage/EmailTriage/Labelling/Web_Labelling/inbox.csv
output_writer = csv.writer(output_file, encoding='utf-8') #, encoding='latin2',utf-8,delimiter=';',

In [11]:
# Folder names Emaila and Category
output_writer.writerow(["email","category"])      

16

In [2]:
mailbox = MailBox('outlook.office365.com')
mailbox.login('officialmailid.com', 'Password', initial_folder='inbox')

In [12]:
email_list_AR = []
 
# get list of email subjects from INBOX folder
mailbox = MailBox('outlook.office365.com')
mailbox.login('afzal.mansury@centelon.com\payments@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='inbox')  # or mailbox.folder.set instead 3d arg

for msg in mailbox.fetch(A(seen=True),mark_seen = False):
    soup = BeautifulSoup(msg.html, features="html.parser")
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    elements = soup.find_all("div", id="Signature")
    #elements = soup.find_all("div","Subject")

    for element in elements:
        element.decompose()

    if len(soup.get_text())>0:
        text = soup.get_text()
    else:
        soup = BeautifulSoup(msg.text)
        text = soup.get_text()
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")

#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    #print("With beautiful soup", text)
    
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    subjects = msg.subject
    print(subjects)
    #print("Origianl mail--------++++++",text)
    
    
    match = re.search(r'((Re\:\sFW\:)|(Re\:)|(RE\:)|(FW\:)|(Fw\:)|(Fwd\:)(RE\:\sFW\:))', subjects)
    #print("------------",text,"-------------")
    # If-statement after search() tests if it succeeded
    if match:
        print('found',match.group(0)) ## 'found word:cat'
        #print(text)
        if match.group()=="FW:" or match.group()=="Fw:" or match.group()=="Fwd:":
            
            try:
                try:
                    pattern = re.compile(r'Subject:')
                    matches = pattern.finditer(text)
                    match_span_sub = []
                    for match in matches:
                        match_span_sub.append(match.span())
                    last_span_sub = match_span_sub[-1][0]
                    print(len(match_span_sub))

                    text = text[last_span_sub:]
                    text_full = []
                    for text in text.splitlines():
                        if not text.startswith(("Subject:")): #"Subject:",
                            text_full.append(text)
                    text = " ".join(text_full)
                except:
                    text = text
                    #print(text)
            except:
                print("no subject found")
            try:
                match = re.search(r'Thanks & Regards,|Thanks & regards,|Kindly regards,|Regards Poonam AR Team|Thanks & Regards|Signing off,|Regards|Best regards,|Kind regards,|Warm regards,|IMPORTANT: This e-mail|CONFIDENTIALITY AND PRIVACY NOTICE|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL', text)
                result = text.index(match.group(0))
                text = text[:result]
                print("Above except condition",text)
                email_list_AR.append(text)
                output_writer.writerow([text,"AR"])
            except:
                print("Not salutations found", text)
                print(text)
                email_list_AR.append(text)
                output_writer.writerow([text,"AR"])
    else:
        #print(text)
        text_full = []
        for text in text.splitlines():
            if not text.startswith(("Subject:","Cc:","Sent:","From:","To:")): #"Subject:",
                text_full.append(text)
                
        text = " ".join(text_full)
        try:
            match = re.search(r'Thanks & Regards,|Thanks & regards,|Kindly regards,|Regards Poonam AR Team|Thanks & Regards|Regards,|IMPORTANT: This e-mail|Best regards,|Warm regards,|CONFIDENTIALITY AND PRIVACY NOTICE|Best Regards|Thanks,|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL', text)
            result = text.index(match.group(0))
            text = text[:result]
            #print(text[:result])
            print(" CISL2021125 mujhe python sikhna hai",text)
            email_list_AR.append(text)
            output_writer.writerow([text,"AR"])
        except:
            print(" CISL2021125 dhundo isse",text)
            email_list_AR.append(text)
            output_writer.writerow([text,"AR"])
    
    print("*"*40,"End of mail body","*"*40)
    
            
output_file.close()



 CISL2021125 mujhe python sikhna hai 
**************************************** End of mail body ****************************************
RE: Payment Receipt
found RE:
**************************************** End of mail body ****************************************
Invoices
 CISL2021125 mujhe python sikhna hai Hi Rob, Going forward request you to mail your monthly invoices to Centelon payments email id( payments@centelon.com). We are trying to streamline processes on payments. 
**************************************** End of mail body ****************************************
RE: Payment Receipt
found RE:
**************************************** End of mail body ****************************************
Your Vodafone email bill for 27-06-2017 to 26-07-2017
 CISL2021125 dhundo isse Vodafone Hi there, Balance $0.00 This bill $50.00DR Total due $50.00DR Direct Debit on 16 Aug 2017 Here's a summary of your spend this month. To view your bill click on 'Download bill' below. 'Pay bill' If you

Your invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd Dear Robin,Please refer updated invoice with payment period.We've attached invoice 012016-00076 for $23,100.00.Payment is due by 17/08/2017.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
Your invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd Dear Robin,We've attached invoice 012016-00079 for $10,395.00 for the period 1-Aug-2017 to 11-Aug- 2017.Payment is due by 17/08/2017.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
We're processing your direct debit
 CISL2021125 mujhe python sikhna hai Vodafone It's that time Hi Manoj, Just letting you know your direct debit is due today and we've started to process your payment. Your payment details: Payment date: 16/08

Your Aug'2017 invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd Dear Team,We've attached Aug'2017 invoice 012016-00080 for $19,855.00.Payment is due by 30/09/2017.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
Your invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd Dear Arun,We've attached invoice 012016-00081 for $39,819.51.Payment is due by 30/09/2017.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
RE: Invoice INV-1552 from West Plaza Business Centre for Centelon Ltd
found RE:
**************************************** End of mail body ****************************************
Re: Invoice INV-1552 from West Plaza Business Centre for Centelon Ltd
found Re:
**************************************** End of mail body *

Atlassian quote AT-42984762
 CISL2021125 dhundo isse Atlassian Here's the quote you requested Hello Manoj, Your quote is now ready and will be valid until 28 Oct 2017. Download quote Simply enter your credit card details to get started immediately or open this quote to view alternative payment options. This link allows anyone to view your quote, including contact information - avoid sharing it publicly. If you still need to make some changes, go to My Atlassian to edit your quote. From there you can also get a proforma invoice and track the status of your order. Cheers, The Atlassians This email was sent to keep you informed about changes to your products or accounts. Privacy Policy • Contact us • Read our Blog • Follow on Twitter Copyright 2017 Atlassian Pty Ltd. All rights reserved. We are located at 341 George Street, Sydney, NSW, 2000, Australia
**************************************** End of mail body ****************************************
Confirmation of your Atlassian order AT

Zoom Payment Processed for Account 220599
 CISL2021125 dhundo isse Dear Ajit Stephen, Thank you for your payment. It has been successfully processed and applied to your account. Below you will find details of the transaction. Zoom Account Number: 220599 Payment Method: Credit Card Payment Date: 10/17/2017 Amount: $14.99 US Dollar Log in at the link below to review or change your billing information or subscription: https://zoom.us/billing Thank you for your business! Zoom Video Communications, Inc. 55 Almaden Boulevard, 6th Floor San Jose, CA 95113 Zoom Support
**************************************** End of mail body ****************************************
FW: Tax Invoice INV35046 from Phone Logic Limited
found FW:
1
Not salutations found Phone Logic Limited 18 Glenreagh Place Torbay Auckland 0630 Ph- 09 473 2925 Fax-09 473 2921 Tax Invoice Centelon Limited West Plaza Business Centre 3 Albert Street Auckland 1010 Invoice Number INV35046 Invoice Date 20/10/2017 GST Number 106-894-965 

...

Your invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AlphaSights Ltd,We've attached invoice 012016-00702 for $100.00.Payment is due by 08/06/2020.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
Skills Academy - A win-win for your team and business
 CISL2021125 dhundo isse A comprehensive learning solution that focuses on addressing skills gaps while supporting the learner's career growth as well as the organization's long-term business strategy. What is a Skills Academy? A comprehensive learning solution that focuses on addressing skills gaps while supporting the learner's career growth as well as the organization's long-term business strategy. This free workbook and guide offers tips on how to: Use 7 key components to build effective Skills Academies Measure the business impact of a Skills Academy in 

Your invoice from Centelon Pty Ltd
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd View full invoice details > View invoice Dear Annie,We've attached invoice 012016-00752 for $19,826.00.Payment is due by 15/09/2020.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
RE: Invoice from WordsMaya for Assessment project
found RE:
**************************************** End of mail body ****************************************
New Speakers Announced: Salesforce, Unilever, and more
 CISL2021125 dhundo isse Join Salesforce's SVP of Global Talent Experience at Forward as she leads a session about leading with resilience through change. Register for free At Forward, our new virtual experience by Udemy for Business, you will get the chance to learn from and connect with L&D, HR and People leaders from all over the world. We have a lot of exciting sessions in store for you to watch when it m

Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00784
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team,We've attached invoice 012016-00784 for $12,546.07. Payment is due by 28/10/2020.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00785
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team,We've attached invoice 012016-00785 for $17,108.32. Payment is due by 28/10/2020.Please get in touch if you've got any questions.
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00786
 CISL2021125 mujhe python sikhna hai MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team,We've attached

Origin's 2020 HSEC Awards contractor category winner!
 CISL2021125 dhundo isse Announcing the winner! The winner of our Best Contractor Innovation and HSEC Excellence category in our 2020 Health, Safety Environment and Community awards is…  FKG! A huge congratulations to FKG – the winner of this year’s Best Contractor Innovation and HSEC Excellence award  and congratulations to the other contractor finalists – Solar Service Guys and Rocky’s Own Transport. FKG work with our Integrated Gas business and have been recognised for supporting and implementing meaningful HSEC initiatives and demonstrating strong a commitment to supporting the local community in regional Queensland. Take a look at the video showcasing FKG’s initiative. Origin is proud of all our contractors and partners who, like FKG, are committed to improving health, safety, environment and community. These awards are held annually, so we’ll let you know when the awards open again in mid-2021. We look forward to again showcas

Re: FW: Due Date Notification 20-NOV-2020
found Re: FW:
**************************************** End of mail body ****************************************
Fwd: NEFT Payment Confirmation Required
 CISL2021125 dhundo isse Get Outlook for Android Dear Customer, Greetings from Tata Tele Business Services! We have received the following credit to TTL account & accordingly request your confirmation of the payment details in the below table for us to enable credit to your account against the respective invoice paid for. UTR NO. Company Name NEFT Amount Account no Posting Amount Invoice Number TDS Value If any IN5BT201116A00PK Centelon It Solutions Llp 88500 605952125 88500 2950375250 Required Request you to regularize this provision in our books by sharing the original TDS certificate immediately. Also going forward, may share these details along with your payments for faster & correct credits to your account. Alternately you can directly pay & credit the invoices by using our self-care porta

In [13]:
len(email_list_AR)

1119

In [16]:
# email_list_AR = []
 
# # get list of email subjects from INBOX folder
# mailbox = MailBox('outlook.office365.com')
# mailbox.login('afzal.mansury@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='test_mail/NAR')  # or mailbox.folder.set instead 3d arg

# for msg in mailbox.fetch(A(seen=True),mark_seen = False):
#     soup = BeautifulSoup(msg.html, features="html.parser")
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")
    
#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    
#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in text.splitlines())
#     # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     # drop blank lines
#     text = '\n'.join(chunk for chunk in chunks if chunk)
#     subjects = msg.subject
#     print(subjects)
    
    
#     match = re.search(r'((Re\:\sFW\:)|(Re\:)|(FW\:)|(RE\:)|(RE\:\sFW\:))', subjects)
#     #print("------------",text,"-------------")
#     # If-statement after search() tests if it succeeded
#     if match:
#         #print('found',match.group(0)) ## 'found word:cat'
#         if match.group()=="FW:":
#             try:
#                 pattern = re.compile(r'Subject:')
#                 matches = pattern.finditer(text)
#                 match_span_sub = []
#                 for match in matches:
#                     match_span_sub.append(match.span())
#                 last_span_sub = match_span_sub[-1][0]
#                 print(last_span_sub)
                
#                 text = text[last_span_sub:]
#                 text_full = []
#                 for text in text.splitlines():
#                     if not text.startswith(("Subject:")): #"Subject:",
#                         text_full.append(text)

#                 text = " ".join(text_full)
#                 try:
#                     match = re.search(r'Thanks & Regards,|Signing off,|Regards|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|THANKING YOU ALL', text)
#                     result = text.index(match.group(0))
#                     text = text[:result]
#                     print(text)
#                     #email_list.append(text)
#                     #output_writer.writerow([text,"AR"])
#                 except:
#                     print(text)
#                     #email_list_AR.append(text)
#                     #output_writer.writerow([text,"AR"]
#             except:
#                 print("______")
#         else:
#             print('found',match.group(0))
#     else:
#         print("Not found")

# For NAR folder
#### Fetching first mail from trial mail including from forwarded mails

In [17]:
output_file = open(r'C:/Users/mansurya/Desktop/Email_triage/test/inbox_NAR_FW.csv','wb')
#C:/Users/mansurya/Desktop/Email_triage/EmailTriage/EmailTriage/Labelling/Web_Labelling/inbox.csv
output_writer = csv.writer(output_file, encoding='utf-8') #, encoding='latin2',utf-8,delimiter=';',

In [18]:
output_writer.writerow(["email","category"])      

16

In [19]:
email_list_NAR = []
 
# get list of email subjects from INBOX folder
mailbox = MailBox('outlook.office365.com')
mailbox.login('afzal.mansury@centelon.com\payments@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='inbox/NAR')  # or mailbox.folder.set instead 3d arg

for msg in mailbox.fetch(A(seen=True),mark_seen = False):
    soup = BeautifulSoup(msg.html, features="html.parser")
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    elements = soup.find_all("div", id="Signature")
    #elements = soup.find_all("div","Subject")

    for element in elements:
        element.decompose()

    if len(soup.get_text())>0:
        text = soup.get_text()
    else:
        soup = BeautifulSoup(msg.text)
        text = soup.get_text()
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")

#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    #print("With beautiful soup", text)
    
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    subjects = msg.subject
    print(subjects)
    
    match = re.search(r'((Re\:\sFW\:)|(Re\:)|(RE\:)|(FW\:)|(Fw\:)|(Fwd\:)(RE\:\sFW\:))', subjects)
    # If-statement after search() tests if it succeeded
    if match:
        print('found',match.group(0)) ## 'found word:cat'
        if match.group()=="FW:" or match.group()=="Fw:" or match.group()=="Fwd:":
            
            try:
                try:
                    pattern = re.compile(r'Subject:')
                    matches = pattern.finditer(text)
                    match_span_sub = []
                    for match in matches:
                        match_span_sub.append(match.span())
                    last_span_sub = match_span_sub[-1][0]
                    print(len(match_span_sub))

                    text = text[last_span_sub:]
                    text_full = []
                    for text in text.splitlines():
                        if not text.startswith(("Subject:")): #"Subject:",
                            text_full.append(text)
                    text = " ".join(text_full)
                except:
                    text = text
                    #print(text)
            except:
                print("no subject found")
            try:
                match = re.search(r'Thanks & Regards,|Thanks & regards,|Kindly regards,|Regards Poonam AR Team|Thanks & Regards|Signing off,|Regards|Kind regards,|Warm regards,|Best regards,|IMPORTANT: This e-mail|CONFIDENTIALITY AND PRIVACY NOTICE|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL', text)
                result = text.index(match.group(0))
                text = text[:result]
                print("Try & Except 1 for forward -------",text)
                email_list_NAR.append(text)
                output_writer.writerow([text,"NAR"])
            except:
                print("Try & Except 1 for forward------", text)
                print(text)
                email_list_NAR.append(text)
                output_writer.writerow([text,"NAR"])
    else:
        #print(text)
        text_full = []
        for text in text.splitlines():
            if not text.startswith(("Subject:","Cc:","Sent:","From:","To:")): #"Subject:",
                text_full.append(text)
                
        text = " ".join(text_full)
        try:
            match = re.search(r'Thanks & Regards,|Kindly regards,|Thanks & regards,|Regards Poonam AR Team|Thanks & Regards|Regards,|IMPORTANT: This e-mail|Warm regards,|Best regards,|CONFIDENTIALITY AND PRIVACY NOTICE|Best Regards|Thanks,|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|Warm Regards,|THANKING YOU ALL', text)
            result = text.index(match.group(0))
            text = text[:result]
            #print(text[:result])
            print(" Try & Except 2 for Subject ----",text)
            email_list_NAR.append(text)
            output_writer.writerow([text,"NAR"])
        except:
            print("Try & Except 2 for Subject ----",text)
            email_list_NAR.append(text)
            output_writer.writerow([text,"NAR"])
    
    print("*"*40,"End of mail body","*"*40)
    
            
output_file.close()


Re: test 
found Re:
**************************************** End of mail body ****************************************
Re: Salesforce.com Invoice 17256036, Acct#6566436, Centelon PTE Ltd
 (Partner Main*)
found Re:
**************************************** End of mail body ****************************************
Unpaid invoice summaries will be sent soon
Try & Except 2 for Subject ---- MYOB Unpaid invoice summaries for August Hi Centelon Pty Ltd, Your customers will receive a statement of their unpaid invoices on the 3rd of August. All invoices with an unpaid status in your MYOB account will appear in the summary. Before we go ahead, make sure: You’ve recorded payments against the right invoices (so we don't send an invoice that's been paid)All invoices are created in MYOB (we can only use the invoices saved in your MYOB software)Your invoices are up to date with the correct details, especially the issue date (unpaid invoices going out this month must be dated before August 2nd) Not rea

Your invoice from Centelon Solutions- Melbourne
 Try & Except 2 for Subject ---- MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Gollogic Pty Ltd, We've attached invoice IV00000000051 for $15,262.50. Payment is due by 31/08/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Bajaj Payment receipt
 Try & Except 2 for Subject ---- For your reference for payment already made. 
**************************************** End of mail body ****************************************
Announcing our keynote speaker…
 Try & Except 2 for Subject ---- Someone special is joining us! Open up to see the surprise... Hi Ann, We’re excited to welcome one of Broadway's biggest stars to the Forward virtual stage — Leslie Odom Jr.! On 15th September, Leslie will take you on a journey through words (and music!) on how Failing Up can unlock your true potential. Sign me up As an a

Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00677
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00677 for $20,097.00. Payment is due by 22/04/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Your invoice from Centelon Solutions- Melbourne
 Try & Except 2 for Subject ---- MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Jennifer, We've attached invoice IV00000000023 for $42,735.00. Payment is due by 15/04/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Your invoice from Centelon Pty Ltd
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear All, We are resending Amey's Jan'2020 for $21,067.20 bas

Payment Remittance: Reference Number - 1115782
Try & Except 2 for Subject ---- Payment Remittance Advice Payment Remittance Advice Mar 20, 2020 To Payee CENTELON PTY LTD LEVEL 3 - 60 CITY RD SOUTHBANK VIC    3006 AUSTRALIA From Payer ORIGIN ENERGY SERVICES LTD GPO BOX 2320 ADELAIDE SA 5001 TELEPHONE :  08 8217 5570 FAX NO. :  08 8217 5555 ABN :  93 007 847 010 The following payment has been remitted to your bank account. Please note, that funds may not be available until the next working business day. Payment Reference Number 1115782 Payment Date Mar 20, 2020 Payment Currency AUD Payment Amount 41,087.20 Bank Account Remitted To 0587188 Remittance Detail Document Reference Number Document Date Document Amount Document Currency Amount Withheld Discount Taken Amount Paid 012016-00632 Jan 31, 2020 13,860.00 AUD .00 13,860.00 012016-00633 Jan 31, 2020 6,160.00 AUD .00 6,160.00 012016-00636 Jan 31, 2020 21,067.20 AUD .00 21,067.20 Total .00 41,087.20 WARNING - This email and any attachments

Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00731
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00731 for $12,100.00. Payment is due by 22/07/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00736
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP Team, We've attached invoice 012016-00736 for $20,575.50. Payment was due by 30/06/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
We're processing your direct debit
 Try & Except 2 for Subject ---- Vodafone We'll be processing your Direct Debit soon Hi Ajit, Just letting you know your direct debit is almost due. Your pa

...

FW: Revised Addition List Required - Reference No: MR-165126 -
 CENTELON - Policy Number :- OG-21-2001-8403-00000001
found FW:
22
Try & Except 1 for forward ------- Dear Ma’am, Request you to kindly provide fresh addition request as the previous one was closed. Thanks & regards, Sasan Kumar Bajaj Allianz General Insurance Company Limited. Mobile:- 9607993275 Email:- Sasan.kumar@bajajallianz.co.in We are providing various digital facilities for our customers to reach out to us at all times: Message us on our social media handles like Twitter (@BajajAllianz), Facebook (@BajajAllianz) &  Instagram (@bajajallianzgeneral) Say Hi on WhatsApp us on 7507245858 for our Always On WhatsApp services Self assistance for youCaringly Yours App, download now http://bit.ly/2vvUbEh If you’ve any grievance, give us a missed call on 8080945060  OR SMS ‘WORRY’ to 575758 , and our Care Specialist will call you backFor global travel customers, they can get claim assistance by calling +91 124 617 4720 Stay Sa

Pay Advice
 Try & Except 2 for Subject ---- Dear Nipun Ojha If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , VIC 3170 T: 1300 853 100 payrollcustomersupport@chandlermacleod.com chandlermacleod.com Part of RGF Staffing We acknowledge and pay our respects, to the Aboriginal and Torres Strait Islander people past, present and emerging, whose land we stand upon today. Chandler Macleod Group partners with the traditional custodians of this land, to Unleash Potential. 
**************************************** End of mail body ****************************************
Pay Advice
 Try & Except 2 for Subject ---- Dear Nipun Ojha If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , 

Your invoice from Centelon Pty Ltd
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear John, We've attached invoice 012016-00718 for $33,550.00. Payment is due by 15/07/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00571
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP Team, We've attached invoice 012016-00571 for $10,725.00. Payment was due by 22/11/2019. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00709
 Try & Except 2 for Subject ---- MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00709 for $12,705.0

Pay Advice
 Try & Except 2 for Subject ---- Dear Niranjan Varadarajan If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , VIC 3170 T: 1300 853 100 payrollcustomersupport@chandlermacleod.com chandlermacleodgroup.com Part of RGF Staffing We acknowledge and pay our respects, to the Aboriginal and Torres Strait Islander people past, present and emerging, whose land we stand upon today. Chandler Macleod Group partners with the traditional custodians of this land, to Unleash Potential. 
**************************************** End of mail body ****************************************
FYI: AAA ENE OPERATING UNIT - Standard Purchase Order 1348460, 1
Try & Except 2 for Subject ---- Oracle Workflow Notification (FYI) From AGUSI, AISHA To payments@centelon.com Sent 12-OCT-20 14:30:30 ID 98126118 Please review the purchase order and any othe

Due Date Notification 20-NOV-2020
Try & Except 2 for Subject ---- `Dear Customer, Greeting from TTL! Please release the payments on priority of below invoice, As per Govt. directive, Telecom and internet service has been classified as essential service. In order ensure smooth operations, we are urging all our customers to make online payment. Your due date is on 20 NOV 2020 and services might be get interrupted after due date due to non-bill payment. If you already paid please share the payment advice and TDS amount also. (From April’2020 Onwards only 2% TDS applicable) Account BCY Invoice Company Name Net Bal 605952125 M03 2165500866 CENTELON IT SOLUTIONS LLP 88500 Please reply back to us if any concern, if already released the payment so please share the details. We urge you to Go Digital and make online payment for your outstanding bills at the earliest, to enjoy uninterrupted services. 1. Online payment gateway for Broadband users: https://billpay.tatatelebroadband.com/BroadbandPor

In [20]:
len(email_list_NAR)

272

In [13]:
# email_list_NAR = []
 
# # get list of email subjects from INBOX folder
# mailbox = MailBox('outlook.office365.com')
# mailbox.login('afzal.mansury@centelon.com\payments@centelon.com', 'mpxbljwxbdzxbvzy', initial_folder='inbox/NAR')  # or mailbox.folder.set instead 3d arg

# for msg in mailbox.fetch(A(seen=True),mark_seen = False):
#     soup = BeautifulSoup(msg.html, features="html.parser")
    
#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out
#     elements = soup.find_all("div", id="Signature")
#     #elements = soup.find_all("div","Subject")
    
#     for element in elements:
#         element.decompose()
#     text = soup.get_text()
    
#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in text.splitlines())
#     # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     # drop blank lines
#     text = '\n'.join(chunk for chunk in chunks if chunk)
#     subjects = msg.subject
#     print(subjects)
    
    
#     match = re.search(r'((Re\:\sFW\:)|(Re\:)|(Fw\:)|(FW\:)|(Fwd\:)|(RE\:)|(RE\:\sFW\:))', subjects)
#     #print("------------",text,"-------------")
#     # If-statement after search() tests if it succeeded
#     if match:
#         print('found',match.group(0)) ## 'found word:cat'
#     else:
#         #print(text)
#         text_full = []
#         for text in text.splitlines():
#             if not text.startswith(("Subject:","Cc:","Sent:","From:","To:")): #"Subject:",
#                 text_full.append(text)
                
#         text = " ".join(text_full)
#         try:
#             match = re.search(r'Thanks & Regards,|Regards,|Kind regards,|CONFIDENTIALITY:|With kind regards,|Kind regards|THANKING YOU ALL|Yours sincerely &  Regards|THANKING YOU ALL', text)
#             result = text.index(match.group(0))
#             text = text[:result]
#             print(text[:result])
#             email_list_NAR.append(text)
#             output_writer.writerow([text,"NAR"])
#         except:
#             print(text)
#             email_list_NAR.append(text)
#             output_writer.writerow([text,"NAR"])
    
#     print("*"*40,"End of mail body","*"*40)
    
            
# output_file.close()
# #Kind regards


# #     match = re.search(r'Subject:\s(RE\:|Re\:|FW\:|Fw\:|Re\:\sFW\:)', text)
# # r'Subject:\s((Re\:\sFW\:)|(Re\:)|(FW\:)|(Fw\:)|(RE\:))|Re\:\sFW\:
# # |Best regards,(.*?)|Best wishes,(.*?)|Fond regards,(.*?)|'
# #                                         r'Kind regards(.*?)|Regards,(.*?)|Sincerely,(.*?)|Sincerely yours,(.*?)|'
# #                                         r'Thank you,(.*?)|With appreciation,(.*?)|Yours sincerely,(.*?)

Re: test 
found Re:
**************************************** End of mail body ****************************************
Re: Salesforce.com Invoice 17256036, Acct#6566436, Centelon PTE Ltd
 (Partner Main*)
found Re:
**************************************** End of mail body ****************************************
Unpaid invoice summaries will be sent soon
MYOB Unpaid invoice summaries for August Hi Centelon Pty Ltd, Your customers will receive a statement of their unpaid invoices on the 3rd of August. All invoices with an unpaid status in your MYOB account will appear in the summary. Before we go ahead, make sure: You’ve recorded payments against the right invoices (so we don't send an invoice that's been paid)All invoices are created in MYOB (we can only use the invoices saved in your MYOB software)Your invoices are up to date with the correct details, especially the issue date (unpaid invoices going out this month must be dated before August 2nd) Not ready yet? If you don’t want to sen

Bajaj Payment receipt
For your reference for payment already made. 
**************************************** End of mail body ****************************************
Announcing our keynote speaker…
Someone special is joining us! Open up to see the surprise... Hi Ann, We’re excited to welcome one of Broadway's biggest stars to the Forward virtual stage — Leslie Odom Jr.! On 15th September, Leslie will take you on a journey through words (and music!) on how Failing Up can unlock your true potential. Sign me up As an award-winning performer in Hamilton and best-selling author of Failing Up: How to Take Risks, Aim Higher, and Never Stop Learning, Leslie will share how he accomplished his goals through failure, perseverance and consistent learning. Leslie’s story will give you the courage and confidence to lead with resilience and move your organisation forward. You won’t want to miss it! 
**************************************** End of mail body ****************************************
We

Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00678
MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00678 for $12,705.00. Payment is due by 22/04/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00677
MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00677 for $20,097.00. Payment is due by 22/04/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Your invoice from Centelon Solutions- Melbourne
MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Jennifer, We've attached invoice IV00000000023 for $42,735.00. Payment is due by 15/04/2020. Please get in touch if you've got an

Payment Remittance: Reference Number - 1115782
Payment Remittance Advice Payment Remittance Advice Mar 20, 2020 To Payee CENTELON PTY LTD LEVEL 3 - 60 CITY RD SOUTHBANK VIC    3006 AUSTRALIA From Payer ORIGIN ENERGY SERVICES LTD GPO BOX 2320 ADELAIDE SA 5001 TELEPHONE :  08 8217 5570 FAX NO. :  08 8217 5555 ABN :  93 007 847 010 The following payment has been remitted to your bank account. Please note, that funds may not be available until the next working business day. Payment Reference Number 1115782 Payment Date Mar 20, 2020 Payment Currency AUD Payment Amount 41,087.20 Bank Account Remitted To 0587188 Remittance Detail Document Reference Number Document Date Document Amount Document Currency Amount Withheld Discount Taken Amount Paid 012016-00632 Jan 31, 2020 13,860.00 AUD .00 13,860.00 012016-00633 Jan 31, 2020 6,160.00 AUD .00 6,160.00 012016-00636 Jan 31, 2020 21,067.20 AUD .00 21,067.20 Total .00 41,087.20 WARNING - This email and any attachments may be private & confidential. 

We're processing your direct debit
Vodafone We'll be processing your Direct Debit soon Hi Ajit, Just letting you know your direct debit is almost due. Your payment details: Payment due date: 13/07/2020 Account number: 879254370 Current amount owing: $50.00 Payment method: Direct Debit - Bank Account Reference: 1240066105 We'll let you know if there's any problem processing your payment. You can also manage your account anytime at My Vodafone or through My Vodafone app*. Thanks, Your Vodafone Team Things you need to know: *Application features are subject to change. The App is free to purchase, however, standard data charges will apply to download it, to access any external links via the App and if used outside Australia. A compatible handset is required and the App is subject to network limitations. Information accessed using the App may not be available at all times or free of errors, omissions or defects. This email was sent to you by Vodafone from a notification only address. Please

...

Your invoice from Centelon Solutions- Melbourne
MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Gollogic Pty Ltd, We've attached invoice IV00000000074 for $22,385.00. Payment is due by 20/10/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Re: Your invoice from Centelon Solutions- Melbourne
found Re:
**************************************** End of mail body ****************************************
Pay Advice
Dear Nipun Ojha If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , VIC 3170 T: 1300 853 100 payrollcustomersupport@chandlermacleod.com chandlermacleodgroup.com Part of RGF Staffing We acknowledge and pay our respects, to the Aboriginal and Torres Strait Islander people past, present and e

**************************************** End of mail body ****************************************
Pay Advice
Dear Nipun Ojha If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , VIC 3170 T: 1300 853 100 payrollcustomersupport@chandlermacleod.com chandlermacleodgroup.com Part of RGF Staffing We acknowledge and pay our respects, to the Aboriginal and Torres Strait Islander people past, present and emerging, whose land we stand upon today. Chandler Macleod Group partners with the traditional custodians of this land, to Unleash Potential. 
**************************************** End of mail body ****************************************
Payment Advice Note from 21.10.2020

**************************************** End of mail body ****************************************
Your invoice from Centelon Pty Ltd
MYOB Centelon Pty Ltd View fu

Your invoice from Centelon Solutions- Melbourne
MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear John, We've attached invoice IV00000000081 for $22,000.00.(Including Invoices team as well) Payment is due by 30/11/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
nvoice Submission – Centelon Pty Ltd - Invoice # 012016-00806
MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00806 for $13,200.00. Payment is due by 24/11/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Invoice Submission – Centelon Pty Ltd - Invoice # 012016-00808
MYOB Centelon Pty Ltd View full invoice details > View invoice Dear AP team, We've attached invoice 012016-00808 for $16,293.64. Payment is due by 24/11/2020. Please g

Your invoice from Centelon Solutions- Melbourne
MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Ched Services, We've attached invoice IV00000000082 for $11,000.00. Payment is due by 30/12/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Your invoice from Centelon Solutions- Melbourne
MYOB Centelon Solutions- Melbourne View full invoice details > View invoice Dear Ched Services, We've attached invoice IV00000000082 for $11,000.00. Payment is due by 30/12/2020. Please get in touch if you've got any questions. 
**************************************** End of mail body ****************************************
Pay Advice
Dear Nipun Ojha If you have any queries regarding your payslip you can email us at PayrollCustomerSupport@chandlermacleod.com or contact us on 1300 853 100. Regards Chandler Macleod Payroll Payroll​ Customer Support Mulgrave , VIC 3170 T

FW: Revised Addition List Required - Reference No: MR-165126 -
 CENTELON - Policy Number :- OG-21-2001-8403-00000001
found FW:
**************************************** End of mail body ****************************************
Your Quarterly Annexure of the Bill (A/C No.605952125)  
Dear Customer, Please find attached your Annexure of the Bill for the month of Nov'20. Your total outstanding due  is Rs.88,500.00 and is due on 20-Dec-20. Hope you have had the opportunity to use our self-care portal - iManage. With iManage you can view, save and pay your bills. iManage also offers you a host of other services, designed for your convenience. To know more about iManage, click hereHow to pay your bill through iManage – Know more 3 Easy Ways to Pay Your Tata Bill: Online payment using iManage:https://www.tatateleservices.com/iManageAuto pay through bank account NEFT/RTGS/ECS/CREDIT CARD: https://www.tatateleservices.com/autopayCheque drop box locations and DD payment: https://www.tatateleservi

In [ ]:
print(len(email_list_AR),len(email_list_NAR))